In [1]:
import pandas as pd
# input data
coro_data = pd.read_csv('coro_data.csv')
coro_data

,Host,Virus,HostTaxID,VirusTaxID,HostNCBIResolved,VirusNCBIResolved,ICTVRatified,HostGenus,HostFamily,HostOrder,...,PMID,ReleaseYear,ReleaseMonth,ReleaseDay,CollectionYear,CollectionMonth,CollectionDay,AssocID,DatabaseDOI,NCBIAccession
0,NaN,alphacoronavirus btcov/ncl_mco1/mop_con/rsa/2012,258862,1532174,True,True,False,mops,molossidae,chiroptera,...,NaN,2014,12.0,12.0,2012.0,NaN,NaN,NaN,NaN,KF843853.1
1,NaN,alphacoronavirus p.sp/k/spain/2007,27671,1076425,True,True,False,pipistrellus,vespertilionidae,chiroptera,...,NaN,2011,8.0,21.0,2007.0,NaN,NaN,NaN,NaN,HQ184060.1
2,NaN,alphacoronavirus sp.,27671,1906673,True,True,False,pipistrellus,vespertilionidae,chiroptera,...,NaN,2017,11.0,28.0,NaN,NaN,NaN,NaN,NaN,LC334391.1
3,NaN,alphacoronavirus tadaridam4,9437,1582306,True,True,False,tadarida,molossidae,chiroptera,...,NaN,2014,12.0,17.0,2011.0,NaN,NaN,NaN,NaN,KM514667.1
4,NaN,avian coronavirus,53274,694014,True,True,True,rhizomys,spalacidae,rodentia,...,NaN,2021,6.0,23.0,2014.0,3.0,19.0,NaN,NaN,MT222079.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35606,vicugna pacos,avian coronavirus,30538,694014,True,True,True,vicugna,camelidae,artiodactyla,...,NaN,2005,3.0,21.0,NaN,NaN,NaN,NaN,NaN,AY939836.2
35607,vicugna pacos,avian coronavirus,30538,694014,True,True,True,vicugna,camelidae,artiodactyla,...,NaN,2005,3.0,22.0,NaN,NaN,NaN,NaN,NaN,AY945840.1
35608,vicugna pacos,betacoronavirus 1,30538,694003,True,True,True,vicugna,camelidae,artiodactyla,...,NaN,2006,NaN,NaN,NaN,NaN,NaN,218960.0,https://doi.org/10.1038/sdata.2015.49,124294673
35609,vicugna pacos,betacoronavirus 1,30538,694003,True,True,True,vicugna,camelidae,artiodactyla,...,NaN,2006,9.0,30.0,NaN,NaN,NaN,NaN,NaN,DQ915164.2


In [4]:
# get coroviridae data with NCBIAccession number
id_accession_data = coro_data[['VirusTaxID','NCBIAccession']]
# remove na
id_accession_data.dropna(inplace=True)

# construct a dictionary, the key of which is ID number and value is NCBIAccession
dic_ID_Accession = {}

for key,value in zip(id_accession_data['VirusTaxID'],id_accession_data['NCBIAccession']):
    process_value = []
    # split the accession number by comma
    for i in value.split(','):
        # remove space
        process_value.append(i.replace(" ",""))
    dic_ID_Accession[key] = dic_ID_Accession.get(key, []) + process_value


<ipython-input-4-890880d44c55>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_accession_data.dropna(inplace=True)


In [33]:
# construct two lists to conserve id and accession
IDs = []
Accessions = []

# construct a two columns data frame
for key, value in dic_ID_Accession.items():
    n_value_count = len(value)
    IDs = IDs + [key]*n_value_count
    Accessions = Accessions + value

coro_id_accession_csv_data = pd.DataFrame(IDs, Accessions)
coro_id_accession_csv_data = coro_id_accession_csv_data.reset_index() # reset the column name
coro_id_accession_csv_data.columns = ['Accession', 'VirusID']

In [34]:
coro_id_accession_csv_data.shape

(1027431, 2)

In [42]:
# remove duplication
coro_id_accession_csv_data = coro_id_accession_csv_data.drop_duplicates(['Accession', 'VirusID'])

# remoce na
coro_id_accession_csv_data.dropna(axis=0,inplace=True)

# remove empty value
coro_id_accession_csv_data = coro_id_accession_csv_data[coro_id_accession_csv_data['Accession'] != '']
coro_id_accession_csv_data.shape

(1027217, 2)

In [43]:
# conserve data into a excel table
coro_id_accession_csv_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv',index=None)

In [44]:
# input coroviridae data with VirusID and Accession
import pandas as pd
coro_id_accession = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv')

In [46]:
# extract the number that represents the NCBI id of url
import re
number = []
accession_data = coro_id_accession["Accession"]

for i in accession_data:
    pattern = re.compile(r"^[0-9]\d+")
    number = number + re.findall(pattern, i)


In [67]:
# seperate data
coro_id_accession = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv')
GI_VirusID_data = coro_id_accession[coro_id_accession.Accession.isin(number)]
Accession_VirusID_data = coro_id_accession[~(coro_id_accession.Accession.isin(number))]
GI_VirusID_data.to_csv('GI_VirusID.csv')
Accession_VirusID_data.to_csv('Acce_VirusID.csv')

In [7]:
### this step can be passed
# get the accession number through the number id
import requests
import re

def get_fasta_info(id):
    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409'.format(id)
    fasta_info = requests.get(fasta_url,timeout=(10,45)).text
    return fasta_info
"""
number_accession = {}
failured_number_id = []
"""
# using WHILE can improve the readablity of code
# the step is designed to convert GI numbers to Accession number, which consumes a lot of time
"""
for i in range(12):
    for id in number[i*100:(i+1)*100]:
        try:
            fasta_info = get_fasta_info(id)
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
        except:
            failured_number_id.append(id)
            print("The {} id numbers are failured to convert to accession".format(id))
            fasta_info = get_fasta_info(failured_number_id[0])
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
            print("The {} id numbers are successed to convert to accession".format(id))
            failured_number_id.pop(0)
"""

The 87244987 id numbers are failured to convert to accession
The 87244987 id numbers are successed to convert to accession
The 182702146 id numbers are failured to convert to accession
The 182702146 id numbers are successed to convert to accession
The 226316432 id numbers are failured to convert to accession
The 226316432 id numbers are successed to convert to accession


In [8]:
# check the length
len(number_accession.keys())

1195

In [16]:
# the step, conserving the data into a csv file, is unnecessary
numberID_accession_data = pd.DataFrame.from_dict(number_accession, orient='index')
numberID_accession_data = numberID_accession_data.reset_index()
numberID_accession_data.columns = ['GI', 'Accession']
numberID_accession_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/GI_Accession.csv',index=None)

In [48]:
numberID_accession_data = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/GI_Accession.csv')

In [19]:
# conserve data
coro_id_accession.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/final_Acce_VirusID.csv',index=None)

In [52]:
renew_data = pd.read_csv('final_Acce_VirusID.csv')
print(renew_data.shape)
renew_data.drop_duplicates(subset=['Accession', 'VirusID'], inplace=True)
print(renew_data.shape)

(1029080, 2)
(1027892, 2)
